In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline

# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm       

from itertools import combinations 

In [79]:
# setActions = ['a1']
# setResponse = [] #....
# setEffect = ['e1', 'e2'] #.... 

header_list = ["nEff", "nResp", "step", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12"]

expFullTable = pd.read_csv('Archive/pmf_tables_for_evaluation_1.csv', ',',  names=header_list, encoding='latin1')

In [80]:
print(expFullTable.head())

columnRes = 'Responses'

expFullTable.count()

   nEff  nResp  step  c1   c2   c3   c4   c5   c6   c7   c8   c9  c10  c11  \
0     4      3   0.2   0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
1     4      3   0.2   0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.2   
2     4      3   0.2   0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.4   
3     4      3   0.2   0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.6   
4     4      3   0.2   0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.8   

   c12  
0  1.0  
1  0.8  
2  0.6  
3  0.4  
4  0.2  


nEff     20000
nResp    20000
step     20000
c1       20000
c2       20000
c3       20000
c4       20000
c5       20000
c6       20000
c7       20000
c8       20000
c9       20000
c10      20000
c11      20000
c12      20000
dtype: int64

In [81]:
print(expFullTable.loc[1])

nEff     4.0
nResp    3.0
step     0.2
c1       0.0
c2       0.0
c3       0.0
c4       1.0
c5       0.0
c6       0.0
c7       0.0
c8       1.0
c9       0.0
c10      0.0
c11      0.2
c12      0.8
Name: 1, dtype: float64


In [82]:



## This function reads the artificially genereted tables from the designed csv
def read_table(exp, row, freq):
    r1 = pd.Series(row)[3:7].tolist();
#     print(r1)
    r2 =pd.Series(row)[7:11].tolist();
#     print(r2)
    r3 = pd.Series(row)[11:15].tolist();
#     print(r3)
    frame = { 'r1': r1, 'r2': r2, 'r3':r3 } 
    tempTable = pd.DataFrame(frame).T
    # for each cells times a 'freq'
    tempTable = tempTable*freq
    return(tempTable)

# create 'table' from CSV
rowNumber = 110
freq = 100
test = read_table(expFullTable, expFullTable.iloc[rowNumber], 100)
print(test)

       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0   0.0  20.0   80.0
r3  80.0  20.0   0.0    0.0


In [83]:


## This function reads the artificially genereted tables from the designed csv
def create_one_response_table(response2effectTable, response):
    oneResponseTable = response2effectTable
    currentR = response

    oneResponseTable.loc['total'] = pd.Series(oneResponseTable.sum())
    oneResponseTable.loc['nonR'] = oneResponseTable.loc['total'] - oneResponseTable.loc[currentR]

    oneResponseTableWithTotal = oneResponseTable
    
    for other_response in myResponses:
        if other_response != currentR:
            oneResponseTable = oneResponseTable.drop(other_response)
    
    oneResponseTable = oneResponseTable.drop('total')
    return(oneResponseTable, oneResponseTableWithTotal)
#     print(oneResponseTable)
    
myResponses = {"r1", "r2", "r3"}
create_one_response_table(test, 'r1')

(         0     1     2      3
 r1     0.0   0.0   0.0  100.0
 nonR  80.0  20.0  20.0   80.0,           0     1     2      3
 r1      0.0   0.0   0.0  100.0
 r2      0.0   0.0  20.0   80.0
 r3     80.0  20.0   0.0    0.0
 total  80.0  20.0  20.0  180.0
 nonR   80.0  20.0  20.0   80.0)

In [84]:


def checking_non_neg_assumptions(oneResponseTable, response2effectTableWithTotal):
    # Checking assumptions
    boolAssumptionTrue = True

    # Assumption 1: For each expected cell, the number > 0
    for aj in response2effectTableWithTotal.columns:
        if response2effectTableWithTotal.at['total', aj] == 0:
            boolAssumptionTrue = False


            
    
    # TODO 
#     if !boolAssumptionTrue:
#         print("NA")

    return(boolAssumptionTrue)


def checking_80perc_expected_assumptions(expected):
    # Assumption 2: At least 80% of expected cells >= 5 // test on the oneResponseTable
    # TO DISCUSS!!! VERY DIFFICULT, checking the full table or checking the one response table. 
    return((expected > 0).sum(1).sum() >= (0.8 * 8))

    


In [90]:
def calculate_number_of_significant_edges(oneResponseTable, currentR):
    countEdgesDraw = 0
    countZeroEdgesDraw = 0
    countSigLessEdgesDraw = 0
    countSigMoreEdgesDraw = 0
    
    stat, p, dof, expected = chi2_cont_row(oneResponseTable)
    
    rtable = sm.stats.Table(oneResponseTable)     
#         print(rtable.resid_pearson)
#         print(rtable.standardized_resids)

    subtable = pd.DataFrame(rtable.standardized_resids)
#     print(subtable)
    
    expectedT = pd.DataFrame(expected, columns = oneResponseTable.columns, index = oneResponseTable.index)
#     print(newTable)
#     print(expectedT)

    for aj in oneResponseTable.columns:
        if (subtable.at[currentR,aj] < -adjresidual) or (subtable.at[currentR,aj] > adjresidual):
            countEdgesDraw +=1
            if(subtable.at[currentR,aj] < -adjresidual):
                countSigLessEdgesDraw += 1
            if (subtable.at[currentR,aj] > adjresidual) :
                countSigMoreEdgesDraw += 1
            if(oneResponseTable.at[currentR, aj] == 0):
                countZeroEdgesDraw +=1
    return(countEdgesDraw, countZeroEdgesDraw, countSigLessEdgesDraw, countSigMoreEdgesDraw)

calculate_number_of_significant_edges(test, 'r1')

(4, 3, 3, 1)

In [89]:

countRow = 0 
for i, row in expFullTable.iterrows():
    if(countRow % 1000 == 0):
        print(countRow)
    countRow+=1
    
    orgtable = read_table(expFullTable, row, freq)
    table = orgtable.copy()
#     print("This is table")
    print(table)
#     print("----")
    
    totNumEdgesOfOurs = 0
    totNumZeroEdgesOurs = 0
    totNumSigMoreEdgesOurs = 0
    
    bool_Assumption_all_responses = True;
    
    pvalues = list()
    numEdges = list()
    numZeroEdges = list()
    numSigMoreEdges = list()
    
    # For each response 
    for response in myResponses:
        oneResponseTable, oneResponseTableWithTotal = create_one_response_table(table, response)
#         print("This is one response:")
#         print(oneResponseTable)
        
        
        # Test assumptions
        bool_Assumption_Single_Response = checking_non_neg_assumptions(oneResponseTable, oneResponseTableWithTotal)
        if(bool_Assumption_Single_Response):
            stat, p, dof, expected = chi2_cont_row(oneResponseTable)
            
            if (checking_80perc_expected_assumptions(expected)):
                pvalues.append(p)
                num, numZero, numSigLess, numSigMore = calculate_number_of_significant_edges(oneResponseTable, response)
                numEdges.append(num)
                numZeroEdges.append(numZero)
                numSigMoreEdges.append(numSigMore)
                
            else:
                bool_Assumption_all_responses = False;
            
        else:
            ## Chi-test's assumptions do not hold. We do not compute anything
            bool_Assumption_all_responses = False;
            
            
            
    if(bool_Assumption_all_responses):
        
        p_adjusted_list = multipletests(pvalues, alpha=.05, method='bonferroni')
        for index, p_adjusted in enumerate(p_adjusted_list[0]):
#             print(p_adjusted)
            if(p_adjusted):
                totNumEdgesOfOurs += numEdges[index]
                totNumZeroEdgesOurs += numZeroEdges[index]
                totNumSigMoreEdgesOurs += numSigMoreEdges[index]
        
        expFullTable.at[i, 'ourNumEdges'] = totNumEdgesOfOurs
        expFullTable.at[i, 'ourNumZeroEdges'] = totNumZeroEdgesOurs
        expFullTable.at[i, 'ourNumPosEdges'] = totNumEdgesOfOurs - totNumZeroEdgesOurs
        expFullTable.at[i, 'ourNumSigMoreEdges'] = totNumSigMoreEdgesOurs
    else:
        expFullTable.at[i, 'ourNumEdges'] = "NA"
        expFullTable.at[i, 'ourNumZeroEdges'] = "NA"
        expFullTable.at[i, 'ourNumPosEdges'] = "NA"
        expFullTable.at[i, 'ourNumSigMoreEdges'] = "NA"
    
    dfgNumEdges = (orgtable > 0).sum(1).sum();
#     print(dfgNumEdges.tolist())
    expFullTable.at[i, 'dfgNumEdges'] = dfgNumEdges
    
    if (countRow > 1000):
        break
    
    print (i, ", our:" , expFullTable.at[i,'ourNumEdges'],
           ", pos:", expFullTable.at[i, 'ourNumPosEdges'], 
           ", sigMore:", expFullTable.at[i, 'ourNumSigMoreEdges'], 
           ", dfg:" , str(expFullTable.at[i,'dfgNumEdges']))
    
    

0
      0    1    2      3
r1  0.0  0.0  0.0  100.0
r2  0.0  0.0  0.0  100.0
r3  0.0  0.0  0.0  100.0
0 , our: NA , pos: NA , sigMore: NA , dfg: 3.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0   0.0  100.0
r3  0.0  0.0  20.0   80.0
1 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0   0.0  100.0
r3  0.0  0.0  40.0   60.0
2 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0   0.0  100.0
r3  0.0  0.0  60.0   40.0
3 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0   0.0  100.0
r3  0.0  0.0  80.0   20.0
4 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0    1      2      3
r1  0.0  0.0    0.0  100.0
r2  0.0  0.0    0.0  100.0
r3  0.0  0.0  100.0    0.0
5 , our: NA , pos: NA , sigMore: NA , dfg: 3.0
      0     1    2      3
r1  0.0   0.0  0.0  100.0
r2  0.0   0.0  0.0  100.0
r3  0.0  20.0 

57 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  20.0   80.0
r3  0.0  0.0  40.0   60.0
58 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  20.0   80.0
r3  0.0  0.0  60.0   40.0
59 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  20.0   80.0
r3  0.0  0.0  80.0   20.0
60 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1      2      3
r1  0.0  0.0    0.0  100.0
r2  0.0  0.0   20.0   80.0
r3  0.0  0.0  100.0    0.0
61 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  20.0   80.0
r3  0.0  20.0   0.0   80.0
62 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  20.0   80.0
r3  0.0  20.0  20.0   60.0
63 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
      0     1     2      3
r

      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  40.0   60.0
r3  0.0  0.0  60.0   40.0
115 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  40.0   60.0
r3  0.0  0.0  80.0   20.0
116 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1      2      3
r1  0.0  0.0    0.0  100.0
r2  0.0  0.0   40.0   60.0
r3  0.0  0.0  100.0    0.0
117 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  40.0   60.0
r3  0.0  20.0   0.0   80.0
118 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  40.0   60.0
r3  0.0  20.0  20.0   60.0
119 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  40.0   60.0
r3  0.0  20.0  40.0   40.0
120 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.

171 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  60.0   40.0
r3  0.0  0.0  80.0   20.0
172 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1      2      3
r1  0.0  0.0    0.0  100.0
r2  0.0  0.0   60.0   40.0
r3  0.0  0.0  100.0    0.0
173 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  60.0   40.0
r3  0.0  20.0   0.0   80.0
174 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  60.0   40.0
r3  0.0  20.0  20.0   60.0
175 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  60.0   40.0
r3  0.0  20.0  40.0   40.0
176 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
      0     1     2      3
r1  0.0   0.0   0.0  100.0
r2  0.0   0.0  60.0   40.0
r3  0.0  20.0  60.0   20.0
177 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
      0     1

223 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  80.0   20.0
r3  0.0  0.0   0.0  100.0
224 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  80.0   20.0
r3  0.0  0.0  20.0   80.0
225 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  80.0   20.0
r3  0.0  0.0  40.0   60.0
226 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  80.0   20.0
r3  0.0  0.0  60.0   40.0
227 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1     2      3
r1  0.0  0.0   0.0  100.0
r2  0.0  0.0  80.0   20.0
r3  0.0  0.0  80.0   20.0
228 , our: NA , pos: NA , sigMore: NA , dfg: 5.0
      0    1      2      3
r1  0.0  0.0    0.0  100.0
r2  0.0  0.0   80.0   20.0
r3  0.0  0.0  100.0    0.0
229 , our: NA , pos: NA , sigMore: NA , dfg: 4.0
      0     1     2      3
r1

...

761 , our: 11 , pos: 6 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  40.0   20.0
r3  20.0  60.0  20.0    0.0
762 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  40.0   20.0
r3  20.0  80.0   0.0    0.0
763 , our: 11 , pos: 5 , sigMore: 4 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  40.0   20.0
r3  40.0   0.0   0.0   60.0
764 , our: 11 , pos: 5 , sigMore: 4 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  40.0   20.0
r3  40.0   0.0  20.0   40.0
765 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  40.0   20.0
r3  40.0   0.0  40.0   20.0
766 , our: 10 , pos: 5 , sigMore: 3 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  40.0   20.0
r3  40.0   0.0  60.0    0.0
767 , our: 11 , pos: 5 , sigMore: 4 , dfg: 6.0

813 , our: 11 , pos: 5 , sigMore: 5 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  60.0    0.0
r3  20.0  40.0   0.0   40.0
814 , our: 9 , pos: 3 , sigMore: 3 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  60.0    0.0
r3  20.0  40.0  20.0   20.0
815 , our: 9 , pos: 4 , sigMore: 3 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  60.0    0.0
r3  20.0  40.0  40.0    0.0
816 , our: 9 , pos: 3 , sigMore: 3 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  60.0    0.0
r3  20.0  60.0   0.0   20.0
817 , our: 11 , pos: 5 , sigMore: 4 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  60.0    0.0
r3  20.0  60.0  20.0    0.0
818 , our: 10 , pos: 4 , sigMore: 4 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  40.0  60.0    0.0
r3  20.0  80.0   0.0    0.0
819 , our: 11 , pos: 4 , sigMore: 4 , dfg: 5.0
  

r3  20.0  20.0  20.0   40.0
867 , our: 11 , pos: 6 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0   0.0   40.0
r3  20.0  20.0  40.0   20.0
868 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0   0.0   40.0
r3  20.0  20.0  60.0    0.0
869 , our: 10 , pos: 4 , sigMore: 4 , dfg: 6.0
       0     1    2      3
r1   0.0   0.0  0.0  100.0
r2   0.0  60.0  0.0   40.0
r3  20.0  40.0  0.0   40.0
870 , our: NA , pos: NA , sigMore: NA , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0   0.0   40.0
r3  20.0  40.0  20.0   20.0
871 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0   0.0   40.0
r3  20.0  40.0  40.0    0.0
872 , our: 10 , pos: 4 , sigMore: 4 , dfg: 6.0
       0     1    2      3
r1   0.0   0.0  0.0  100.0
r2   0.0  60.0  0.0   40.0
r3  20.0  60.0  0.0   20.0
873 , our: NA , pos: NA 

920 , our: 11 , pos: 6 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  20.0   20.0
r3  20.0   0.0  80.0    0.0
921 , our: 12 , pos: 6 , sigMore: 4 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  20.0   20.0
r3  20.0  20.0   0.0   60.0
922 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  20.0   20.0
r3  20.0  20.0  20.0   40.0
923 , our: 8 , pos: 4 , sigMore: 3 , dfg: 8.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  20.0   20.0
r3  20.0  20.0  40.0   20.0
924 , our: 10 , pos: 6 , sigMore: 4 , dfg: 8.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  20.0   20.0
r3  20.0  20.0  60.0    0.0
925 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  20.0   20.0
r3  20.0  40.0   0.0   40.0
926 , our: 10 , pos: 5 , sigMore: 4 , dfg: 7.0


       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  40.0    0.0
r3  20.0   0.0  20.0   60.0
974 , our: 11 , pos: 5 , sigMore: 5 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  40.0    0.0
r3  20.0   0.0  40.0   40.0
975 , our: 9 , pos: 3 , sigMore: 3 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  40.0    0.0
r3  20.0   0.0  60.0   20.0
976 , our: 11 , pos: 5 , sigMore: 4 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  40.0    0.0
r3  20.0   0.0  80.0    0.0
977 , our: 11 , pos: 4 , sigMore: 4 , dfg: 5.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  40.0    0.0
r3  20.0  20.0   0.0   60.0
978 , our: 11 , pos: 5 , sigMore: 5 , dfg: 6.0
       0     1     2      3
r1   0.0   0.0   0.0  100.0
r2   0.0  60.0  40.0    0.0
r3  20.0  20.0  20.0   40.0
979 , our: 9 , pos: 4 , sigMore: 4 , dfg: 7.0
       0     1     2      3
r1   0.0   0.0   0.0

In [91]:
expFullTable.head()
df.to_csv
expFullTable = pd.read_csv('Archive/pmf_tables_for_evaluation_1_res.csv', ',',  names=header_list, encoding='latin1')

,nEff,nResp,step,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,ourNumEdges,ourNumZeroEdges,ourNumPosEdges,dfgNumEdges,ourNumSigMoreEdges
0,4,3,0.2,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NA,NA,NA,3.0,NA
1,4,3,0.2,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.2,0.8,NA,NA,NA,4.0,NA
2,4,3,0.2,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.4,0.6,NA,NA,NA,4.0,NA
3,4,3,0.2,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.6,0.4,NA,NA,NA,4.0,NA
4,4,3,0.2,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.8,0.2,NA,NA,NA,4.0,NA


(         0     1     2      3
 r1     0.0   0.0   0.0  100.0
 nonR  80.0  20.0  20.0   80.0,           0     1     2      3
 r1      0.0   0.0   0.0  100.0
 r2      0.0   0.0  20.0   80.0
 r3     80.0  20.0   0.0    0.0
 total  80.0  20.0  20.0  180.0
 nonR   80.0  20.0  20.0   80.0)

In [114]:
# oneResponseTable = oneResponseTable.drop('r2')
# oneResponseTable = oneResponseTable.drop('r3')
# oneResponseTable = oneResponseTable.drop('total')
# oneResponseTable

,0,1,2,3
r1,0.0,0.0,0.0,100.0
nonR,80.0,20.0,20.0,80.0


NameError: name 'test' is not defined

In [8]:
# pvalues = list() # keep a list of computed p values to use for adjusting p-values
# countEdgesDraw=0

# adjresidual = 2.57
# newTable = oneResponseTable

# stat, p, dof, expected = chi2_cont_row(newTable)
    
# if (p <= 0.05):
#     rtable = sm.stats.Table(newTable)     
# #         print(rtable.resid_pearson)
# #         print(rtable.standardized_resids)

#     subtable = pd.DataFrame(rtable.standardized_resids)
#     print(subtable)
#     expectedT = pd.DataFrame(expected, columns = newTable.columns, index = newTable.index)
#     print(newTable)
#     print(expectedT)

#     for aj in table.columns:
#         if (subtable.at[currentR,aj] < -adjresidual) or (subtable.at[currentR,aj] > adjresidual):
#             countEdgesDraw +=1

            
            
# pvalues.append(p)
# print(countEdgesDraw)
                
    
# print(pvalues)
# p_adjusted = multipletests(pvalues, alpha=.05, method='bonferroni')
# # Print the resulting conclusions
# print(p_adjusted[0])

# # Print the adjusted p-values themselves 
# print(p_adjusted[1])
# print("////////////")

NameError: name 'oneResponseTable' is not defined

0
0
skip
1
1
skip
2
2
skip
3
3
skip
4


In [9]:
# import graphviz
# import pygraphviz as pgv

# # chi-squared test with similar proportions
# from scipy.stats import chi2_contingency
# from scipy.stats import chi2
# from scipy import stats
# from statsmodels.stats.proportion import proportions_ztest
# from statsmodels.sandbox.stats.multicomp import multipletests
# import statsmodels.api as sm       

# from itertools import combinations 

# def chi2_cont_row(df):
#     stat, p, dof, expected = chi2_contingency(df, correction=True)
# #     print('dof=%d' % dof)
# #     print(expected)

#     # interpret test-statistic
#     prob = 0.95
#     critical = chi2.ppf(prob, dof)
# #     print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

# #     if abs(stat) >= critical:
# #         print('Dependent (reject H0)')
# #     else:
# #         print('Independent (fail to reject H0)')


#     # interpret p-value
#     alpha = 1.0 - prob
# #     print('significance=%.3f, p=%.3f' % (alpha, p))
# #     if p <= alpha:
# #         print('Dependent (reject H0)')
# #     else:
# #         print('Independent (fail to reject H0)')
#     return stat, p, dof, expected 